In [ ]:
import os
from pathlib import Path
import sys
import h5py
from collections import defaultdict

from third_party.hloc.hloc import extract_features, extractors, matchers, pairs_from_retrieval, match_features, visualization
from third_party.hloc.hloc.extract_features import ImageDataset
from third_party.hloc.hloc.localize_sfm import QueryLocalizer, pose_from_cluster
from third_party.hloc.hloc.fast_localize import localize
from third_party.hloc.hloc.utils import viz_3d, io
from third_party.hloc.hloc.utils.base_model import dynamic_load
from third_party.hloc.hloc.utils.io import list_h5_names
from third_party.hloc.hloc.utils.parsers import names_to_pair

import pycolmap
import numpy as np
from scipy.spatial.transform import Rotation
import torch

In [ ]:
LOCAL_FEATURE_EXTRACTOR = 'superpoint_aachen'
GLOBAL_DESCRIPTOR_EXTRACTOR = 'netvlad'
MATCHER = 'superglue'

In [ ]:
# img_path = '/home/sagar/Repos/spatial-server/data/query_data/Cubicles/d6a0ab62-900e-4b92-8aa8-558e2c888a26/query_image.png' # Lab Front
img_path = '/code/data/query_data/Area2300NoYCheck/e3c01aa7-beed-4c48-81b0-dc235f132ba6/query_image.png' # Cubicles
dataset_name = 'Area2300NoYCheck'
dataset_path = f'/code/data/map_data/{dataset_name}/hloc_data'

# Slow localization

In [ ]:
%%time

img_path = Path(img_path)

local_feature_conf = extract_features.confs[LOCAL_FEATURE_EXTRACTOR]
global_descriptor_conf = extract_features.confs[GLOBAL_DESCRIPTOR_EXTRACTOR]
match_features_conf = match_features.confs[MATCHER]

dataset = Path(dataset_path)
db_global_descriptors_path = (dataset / global_descriptor_conf['output']).with_suffix('.h5')
db_local_features_path = (dataset / local_feature_conf['output']).with_suffix('.h5')
# Use the scaled reconstruction if it exists
db_reconstruction = dataset / 'scaled_sfm_reconstruction'
if not db_reconstruction.exists():
    db_reconstruction = dataset / 'sfm_reconstruction'
db_reconstruction = dataset / 'sfm_reconstruction'
db_image_dir = dataset.parents[0] / 'ns_data' / 'images'

query_image_name = os.path.basename(img_path)

# Query data
query_processing_data_dir = Path(os.path.dirname(img_path))
query_global_matches_path = query_processing_data_dir / 'global_match_pairs.txt'
query_local_match_path = query_processing_data_dir / 'local_match_data.h5'
query_results = query_processing_data_dir / 'query_results.txt'

# Extarct local features and global descriptor for the new image
query_local_features_path = extract_features.main(
    conf = local_feature_conf,
    image_dir = query_processing_data_dir,
    export_dir = query_processing_data_dir,
    image_list = [query_image_name]
)

query_global_descriptor_path = extract_features.main(
    conf = global_descriptor_conf,
    image_dir = query_processing_data_dir,
    export_dir = query_processing_data_dir,
    image_list = [query_image_name]
)

## Use global descriptor matching to get candidate matches
nearest_candidate_images = pairs_from_retrieval.save_global_candidates_for_query(
    db_descriptors = db_global_descriptors_path,
    query_descriptor = query_global_descriptor_path,
    query_image_names = [query_image_name],
    num_matched = 10,
    output_file_path = query_global_matches_path
)

## Match the query image against the candidate pairs from above
match_features.match_from_paths(
    conf = match_features_conf,
    pairs_path = query_global_matches_path,
    match_path = query_local_match_path,
    feature_path_q = query_local_features_path,
    feature_path_ref = db_local_features_path
)

## Now we have global candidate and thier mathces. We use this, along with SfM reconstruction to localize the image.
reconstruction = pycolmap.Reconstruction(db_reconstruction.__str__())
camera = pycolmap.infer_camera_from_image(query_processing_data_dir / query_image_name)
ref_ids = [reconstruction.find_image_with_name(r).image_id for r in nearest_candidate_images]
conf = {
    'estimation': {'ransac': {'max_error': 12}},
    'refinement': {'refine_focal_length': True, 'refine_extra_params': True},
}
localizer = QueryLocalizer(reconstruction, conf)
ret, log = pose_from_cluster(
    localizer = localizer, 
    qname = query_image_name, 
    query_camera = camera, 
    db_ids = ref_ids, 
    features_path = db_local_features_path, 
    matches_path = query_local_match_path,
    features_q_path = query_local_features_path
)

print('Num_inliers: ', ret['num_inliers'])

In [ ]:
confidence = log['PnP_ret']['num_inliers'] / log['keypoints_query'].shape[0]
print("Confidence: ", confidence)

In [ ]:
visualization.visualize_loc_from_log(
    image_dir = query_processing_data_dir, 
    query_name = query_image_name, 
    loc = log, 
    reconstruction = reconstruction, 
    db_image_dir = db_image_dir
)

In [ ]:
# fig = viz_3d.init_figure()
# viz_3d.plot_reconstruction(fig, reconstruction, color='rgba(255,0,0,0.5)', points_rgb=True)
# pose = pycolmap.Image(tvec=ret['tvec'], qvec=ret['qvec'])
# # viz_3d.plot_camera_colmap(fig, pose, camera, color='rgba(0,255,0,0.5)', name=query_image_name, fill=True)
# viz_3d.plot_camera_colmap(fig, pose, reconstruction.cameras[1], color='rgba(0,255,0,0.5)', name=query_image_name, fill=True)
# # visualize 2D-3D correspodences
# inl_3d = np.array([reconstruction.points3D[pid].xyz for pid in np.array(log['points3D_ids'])[ret['inliers']]])
# viz_3d.plot_points(fig, inl_3d, color="lime", ps=1, name=query_image_name)
# fig.show()

In [ ]:
import plotly.graph_objects as go

fig = viz_3d.init_figure()
viz_3d.plot_reconstruction(fig, reconstruction, color='rgba(255,0,0,0.5)', points_rgb=True, cameras = False)

# Add the camera
pose = pycolmap.Image(tvec=ret['tvec'], qvec=ret['qvec'])
viz_3d.plot_camera_colmap(fig, pose, reconstruction.cameras[1], color='rgba(255,255,255,1)', name=query_image_name, fill=True)

# visualize 2D-3D correspodences
inl_3d = np.array([reconstruction.points3D[pid].xyz for pid in np.array(log['points3D_ids'])[ret['inliers']]])
viz_3d.plot_points(fig, inl_3d, color="lime", ps=1, name=query_image_name)

# X axis
fig.add_trace(go.Scatter3d(
    x = [0,10],
    y = [0,0],
    z = [0,0],
    line=dict(
        color='red',
        width=2
    )
))

# Y axis
fig.add_trace(go.Scatter3d(
    x = [0,0],
    y = [0,10],
    z = [0,0],
    line=dict(
        color='green',
        width=2
    )
))

# Z axis
fig.add_trace(go.Scatter3d(
    x = [0,0],
    y = [0,0],
    z = [0,10],
    line=dict(
        color='blue',
        width=2
    )
))
    
# Add line
# for i in range(0,10):
#     tvec = [0, i, 0]
#     rvec = [45, 0, 0]
#     qvec = Rotation.from_euler('xyz', rvec, degrees = True).as_quat()
#     fake_camera = reconstruction.cameras[1]
#     fake_pose = pycolmap.Image(tvec=tvec, qvec=qvec)
#     viz_3d.plot_camera_colmap(fig, fake_pose, fake_camera, color='rgba(255,255,255,1)', name='z', fill=True)

fig.show()

In [ ]:
from scipy.spatial.transform import Rotation
import numpy as np
np.set_printoptions(precision=3, suppress = True)

def simplify_small_values(arr, tol = 1e-10):
    arr[np.abs(arr) < tol] = 0
    return arr

def homogenize(rotation, translation):
    """
    Combine the (3,3) rotation matrix and (3,) translation matrix to
    one (4,4) transformation matrix
    """
    homogenous_array = np.eye(4)
    homogenous_array[:3, :3] = rotation
    homogenous_array[:3, 3] = translation
    return homogenous_array

def rot_from_qvec(qvec):
    # Change (w,x,y,z) to (x,y,z,w)
    return Rotation.from_quat([qvec[1], qvec[2], qvec[3], qvec[0]])

In [ ]:
import numpy as np
import os
from pathlib import Path
import pickle

from scipy.spatial.transform import Rotation

def convert_hloc_to_blender_frame(matrix):
    # Add 180 degrees to X - change in convention
    matrix = np.array(matrix)
    euler_xyz = Rotation.from_matrix(matrix[:3, :3]).as_euler("xyz", degrees = True)
    euler_xyz[0] += 180
    rotmat = Rotation.from_euler('xyz', euler_xyz, degrees = True).as_matrix()
    matrix[:3, :3] = rotmat
    return matrix

def convert_blender_to_aframe_frame(matrix):
    # Rotate -90 degrees along x-axis
    T_B_to_A = np.eye(4)
    T_B_to_A[:3,:3] = Rotation.from_euler('xyz', [-90,0,0], degrees = True).as_matrix()
    return T_B_to_A @ matrix

def get_arscene_pose_matrix(aframe_camera_pose, hloc_camera_matrix, dataset_name):
    blender_camera_matrix = convert_hloc_to_blender_frame(hloc_camera_matrix)
    blender_camera_matrix_in_aframe = convert_blender_to_aframe_frame(blender_camera_matrix)

    aframe_camera_matrix = np.array(aframe_camera_pose).reshape((4,4)).T

    arscene_pose_aframe = aframe_camera_matrix @ np.linalg.inv(blender_camera_matrix_in_aframe)

    return arscene_pose_aframe.T.flatten().tolist()

In [ ]:
hloc_camera_matrix = np.linalg.inv(homogenize(rot_from_qvec(ret['qvec']).as_matrix(), ret['tvec']))
blender_camera_matrix = convert_hloc_to_blender_frame(hloc_camera_matrix)
blender_camera_matrix_in_aframe = convert_blender_to_aframe_frame(blender_camera_matrix)

In [ ]:
blender_camera_matrix_in_aframe.T.flatten()

In [ ]:
# Read aframe camera matrix from the saved file
aframe_camera_file = Path(img_path).parent / 'aframe_camera_matrix_world.pkl'
with open(aframe_camera_file, 'rb') as f:
    aframe_camera_matrix = pickle.load(f)
    aframe_camera_matrix = np.array(aframe_camera_matrix).reshape((4,4)).T

In [ ]:
arscene_pose_aframe = aframe_camera_matrix @ np.linalg.inv(blender_camera_matrix_in_aframe) #@ convert_blender_to_aframe_frame(np.eye(4))
arscene_pose_aframe.T.flatten()

# Fast localization

Things needed in memory:
- Image
- Local feature extractor (Superpoint)
- Global descriptor (NetVLad)
- Reconstruction

In [ ]:
%%time

# Load global memory - Common data across all maps

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Load local feature extractor model (Superpoint)
local_feature_conf = extract_features.confs[LOCAL_FEATURE_EXTRACTOR]
Model = dynamic_load(extractors, local_feature_conf['model']['name'])
local_features_extractor_model = Model(local_feature_conf['model']).eval().to(device)

# Load global descriptor model (Netvlad)
global_descriptor_conf = extract_features.confs[GLOBAL_DESCRIPTOR_EXTRACTOR]
Model = dynamic_load(extractors, global_descriptor_conf['model']['name'])
global_descriptor_model = Model(global_descriptor_conf['model']).eval().to(device)

# Load matcher model (SuperGlue)
match_features_conf = match_features.confs[MATCHER]
Model = dynamic_load(matchers, match_features_conf['model']['name'])
matcher_model = Model(match_features_conf['model']).eval().to(device)

In [ ]:
%%time
# Load map specific memory

# Define database paths
dataset = Path(dataset_path)
db_local_features_path = (dataset / local_feature_conf['output']).with_suffix('.h5')

db_reconstruction = dataset / 'scaled_sfm_reconstruction'
if not db_reconstruction.exists():
    db_reconstruction = dataset / 'sfm_reconstruction'

# Load global descriptors from the database
db_global_descriptors_path = (dataset / global_descriptor_conf['output']).with_suffix('.h5')
db_image_names = np.array(list_h5_names(db_global_descriptors_path))
db_global_descriptors = pairs_from_retrieval.get_descriptors(db_image_names, db_global_descriptors_path)
db_global_descriptors = db_global_descriptors.to(device)

In [ ]:
%%time

# Localize at query time
# Query data
img_path = Path(img_path)
query_image_name = os.path.basename(img_path)
query_processing_data_dir = Path(os.path.dirname(img_path))

ret_new, log_new = localize(
    query_processing_data_dir = query_processing_data_dir, 
    query_image_name = query_image_name, 
    device = device, 
    local_feature_conf = local_feature_conf, 
    local_features_extractor_model = local_features_extractor_model, 
    global_descriptor_conf = global_descriptor_conf, 
    global_descriptor_model = global_descriptor_model, 
    db_global_descriptors = db_global_descriptors, 
    db_image_names = db_image_names,
    db_local_features_path = db_local_features_path,
    matcher_model = matcher_model,
    db_reconstruction = db_reconstruction
)